In [ ]:
#BELOW DOWN IS THE COMPLETE AUTOMATED PI CHART GENERATING

In [ ]:
Critical High Medium Low Info Total
192.168.1.101 35 143 29 3 0 210
192.168.1.103 12 22 14 3 0 51
192.168.1.111 29 92 24 3 0 148
192.168.1.136 25 80 18 4 0 127
192.168.1.152 21 81 29 3 0 134
192.168.1.183 35 72 21 4 0 132

In [ ]:
ABOVE UP SIDE IS THE SAMPLE TEXT INPUT WHICH ARE COPY FROM THE REPORTS SAME THIS TEXT IS COPIED FROM THE TABLES 

In [2]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- 1. VISUAL CONFIGURATION ---
COLORS = {
    'Critical': '#C00000',  # Deep Dark Red
    'High':     '#FF0000',  # Bright Red
    'Medium':   '#FFC000',  # Amber/Yellow
    'Low':      '#92D050'   # Light Green
}
TABLE_COLORS = [COLORS['Critical'], COLORS['High'], COLORS['Medium'], COLORS['Low']]
TEXT_COLORS = ['white', 'white', 'black', 'black']

# --- 2. DATA PROCESSING FUNCTION ---
def process_pasted_data(raw_text):
    """
    Parses the pasted text to calculate totals.
    Expected format: IP Address | Crit | High | Med | Low | Info | Total
    """
    crit_sum = 0
    high_sum = 0
    med_sum = 0
    low_sum = 0
    
    lines = raw_text.strip().split('\n')
    
    for line in lines:
        parts = line.split()
        
        # We need at least 5 columns (IP + 4 Severities)
        if len(parts) < 5:
            continue
            
        # Try to parse the columns. 
        # We assume the standard structure: [IP] [Crit] [High] [Med] [Low] ...
        # If the first column is an IP, the counts start at index 1.
        try:
            # Check if index 1 is a number (Skipping headers like "Critical High...")
            if not parts[1].isdigit():
                continue
                
            crit_sum += int(parts[1])
            high_sum += int(parts[2])
            med_sum  += int(parts[3])
            low_sum  += int(parts[4])
            
        except ValueError:
            continue # Skip lines that don't look like data

    return [crit_sum, high_sum, med_sum, low_sum]

# --- 3. GENERATE FUNCTION ---
def generate_dashboard(b):
    with out_plot:
        clear_output(wait=True)
        vessel_name = w_name.value.strip()
        raw_data = w_data.value.strip()
        
        if not vessel_name:
            print("⚠️ Error: Please enter a Vessel Name.")
            return
        
        if not raw_data:
            print("⚠️ Error: Please paste the report data.")
            return

        # PROCESS THE DATA
        counts = process_pasted_data(raw_data)
        labels = ['Critical', 'High', 'Medium', 'Low']
        
        if sum(counts) == 0:
            print("⚠️ Error: No valid data found. Check your copy-paste.")
            print("Expected format per line: 192.168.x.x  35  143  29  3 ...")
            return

        # --- PLOTTING ---
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
        fig.suptitle(vessel_name.upper(), fontsize=20, fontweight='bold', x=0.15, y=0.95)

        # 1. PIE CHART
        wedges, texts = ax1.pie(
            counts, labels=counts, colors=TABLE_COLORS,
            startangle=90, counterclock=False,
            textprops={'fontsize': 12, 'weight': 'bold'}
        )
        ax1.set_aspect('equal')
        ax1.legend(wedges, labels, title="Severity", loc="center left", bbox_to_anchor=(0.9, 0, 0.5, 1))

        # 2. TABLE
        ax2.axis('off')
        col_labels = ['Severity', 'Count']
        cell_text = [[labels[i], counts[i]] for i in range(4)]

        the_table = ax2.table(
            cellText=cell_text, colLabels=col_labels,
            loc='center', cellLoc='center', colColours=['black', 'black']
        )

        # Table Styling
        the_table.auto_set_font_size(False)
        the_table.set_fontsize(12)
        the_table.scale(0.8, 2.5)

        for (row, col), cell in the_table.get_celld().items():
            cell.set_edgecolor('white')
            cell.set_linewidth(2)
            if row == 0:
                cell.set_facecolor('black')
                cell.set_text_props(color='white', weight='bold', fontsize=14)
            else:
                idx = row - 1
                cell.set_facecolor(TABLE_COLORS[idx])
                cell.set_text_props(color=TEXT_COLORS[idx], weight='bold', fontsize=13)

        # Save and Show
        filename = f"{vessel_name.replace(' ', '_')}_PI_CHART.png"
        plt.tight_layout()
        plt.subplots_adjust(wspace=0.1)
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        print(f"✅ Generated: {filename}")
        plt.show()

# --- 4. GUI WIDGETS ---
def clear_form(b):
    w_name.value = ''
    w_data.value = ''
    with out_plot: clear_output()

# New Layout
lbl_header = widgets.HTML("<h2>📊 VAPT Auto-Calculator & Generator</h2>")
w_name = widgets.Text(description='Vessel Name:', placeholder='CORAL PEARL', layout=widgets.Layout(width='50%'))

# The Paste Box
w_data = widgets.Textarea(
    description='Paste Data:',
    placeholder='Paste your rows here (e.g., 192.168.1.101 35 143...)',
    layout=widgets.Layout(width='98%', height='150px')
)

btn_gen = widgets.Button(description='Calculate & Generate', button_style='success', icon='gears', layout=widgets.Layout(width='200px'))
btn_gen.on_click(generate_dashboard)

btn_clear = widgets.Button(description='Clear', button_style='warning', icon='trash', layout=widgets.Layout(width='100px'))
btn_clear.on_click(clear_form)

ui = widgets.VBox([
    lbl_header,
    widgets.HTML("<b>1. Vessel Name:</b>"), w_name,
    widgets.HTML("<br><b>2. Paste Report Data (Directly from text/excel):</b>"), w_data,
    widgets.HTML("<br>"),
    widgets.HBox([btn_gen, btn_clear]),
    widgets.Output(layout={'border': '1px solid #ddd'})
])
out_plot = ui.children[-1]

display(ui)